<h2><center>Walmart Sales Forecasting</center></h2>

In [1]:
import pandas as pd 
from downcast import reduce
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Env variables
non_day_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
level_groupings = {1: [],   #Day sales accorss everything. [Results in single row with column of each day sales.]
                   2: ["state_id"], 
                   3: ["store_id"], 
                   4: ["cat_id"], 
                   5: ["dept_id"], 
                   6: ["state_id", "cat_id"], 
                   7: ["state_id", "dept_id"], 
                   8: ["store_id", "cat_id"], 
                   9: ["store_id", "dept_id"],
                   10:["item_id"], 
                   11:["item_id", "state_id"],
                   12:[]  #["item_id", "store_id"] == 'id'
                   }
project_path = "~/Desktop/workspace/Walmart_Sales_Deployment/"
data_dir = project_path+"dataset/"
processed_fpath = project_path+'processing_data/'
submission_dir = project_path+'outputs/'

random_state = 11223300

In [ ]:
sales=pd.read_csv(data_dir+'sales_train_evaluation.csv')
calendar=pd.read_csv(data_dir+'calendar.csv')
prices=pd.read_csv(data_dir+'sell_prices.csv')

# This will take some time
sales=reduce(sales)
calendar=reduce(calendar)
prices=reduce(prices)

### Feature Engineering 

In [ ]:
# If day is weekend or not
calendar['Weekend'] = calendar['weekday'].apply(lambda day: 1 if day.lower() == 'saturday' or day.lower() == 'sunday' else 0).astype(np.int8)

In [ ]:
# Get day of the Month from data object
calendar['Day_Of_Month'] = calendar['date'].dt.day.astype(np.int8)

In [ ]:
# OnehotEncoding for event types
OHE_df = pd.get_dummies(calendar[['event_type_1','event_type_1']], prefix=['ET1', 'ET2'],dtype = np.int8)
cols = OHE_df.columns
OHE_df['total_events'] = OHE_df[cols].sum(axis = 1).astype(np.int8)
calendar = pd.concat([calendar,OHE_df],axis=1)
del OHE_df
del cols
calendar.sample(2)

In [ ]:
# Drop reduandant columns
calendar.drop(['date','weekday','wday','event_name_1','event_type_1','event_name_2','event_type_2'],axis=1,inplace=True)

In [ ]:
calendar = reduce(calendar)
calendar.head()

In [ ]:
sales[['id','item_id','dept_id','cat_id','store_id','state_id']].nunique()

In [ ]:
# Merge
sales=pd.melt(sales,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='units_sold')
sales=pd.merge(sales,calendar,on='d',how='left')
sales=pd.merge(sales,prices,on=['item_id','store_id','wm_yr_wk'],how='left')
sales.head()

In [ ]:
# No need after melt & merging
del calendar
del prices

In [ ]:
# To impute the NaN's of sell_prices, we will take average prices of product grouped.
sales['sell_price']=sales['sell_price'].fillna(sales.groupby('id')['sell_price'].transform('mean'))
sales.isna().sum()

In [ ]:
#Store the categories along with their codes
map_id = dict(zip(sales.id.cat.codes, sales.id))
map_item_id = dict(zip(sales.item_id.cat.codes, sales.item_id))
map_dept_id = dict(zip(sales.dept_id.cat.codes, sales.dept_id))
map_cat_id = dict(zip(sales.cat_id.cat.codes, sales.cat_id))
map_store_id = dict(zip(sales.store_id.cat.codes, sales.store_id))
map_state_id = dict(zip(sales.state_id.cat.codes, sales.state_id))

joblib.dump(map_id, processed_fpath+'map_id.pkl')

In [ ]:
# Labelling the categories provided.
category=['id','item_id','dept_id','cat_id','store_id','state_id']
for cat in tqdm(category):
    sales[cat] = sales[cat].cat.codes
sales.head()

In [ ]:
sales['d']  = sales['d'].apply(lambda s: int(s.split('_')[1]))
sales = reduce(sales)
sales.info()

In [ ]:
intermediate_df = sales.groupby(['id'])['units_sold']

In [ ]:
# Rolling Window Statistics
window_size = [7,14,21,28,35,42,49,56,63] 
for window in tqdm(window_size):
    sales['mean_units_rolling_'+str(window)] = intermediate_df\
                                .transform(lambda x: x.rolling(window=window).mean())\
                                .fillna(0)\
                                .astype(np.float16)
    
sales.info()

In [ ]:
# Lags
most_useful_lags = [7,14,21,28,35,42,70]
for lag in tqdm(most_useful_lags):
    sales['lag_'+str(lag)] = intermediate_df.shift(lag).fillna(0)
    
sales.info()

In [ ]:
# Rolling window median and Standard Deviation
window_size = [7,14,21,28,35,42,70] 
for window in window_size:
    sales['median_units_rolling_'+str(window)] = intermediate_df\
                                    .transform(lambda x: x.rolling(window=window).median())\
                                    .astype(np.float16)
    sales['std_units_rolling_'+str(window)] = intermediate_df\
                                    .transform(lambda x: x.rolling(window=window).std())\
                                    .astype(np.float16)

sales.fillna(0, inplace=True)
sales.info()

In [ ]:
# Expanding Window Stats
for window in window_size:
    sales['mean_units_expanding_'+str(window)] = intermediate_df\
                                      .transform(lambda x: x.expanding(window).mean())\
                                      .astype(np.float16)

    sales['median_units_expanding_'+str(window)] = intermediate_df\
                                      .transform(lambda x: x.expanding(window).median())\
                                      .astype(np.float16)

    sales['std_units_expanding_'+str(window)] = intermediate_df\
                                      .transform(lambda x: x.expanding(window).median())\
                                      .astype(np.float16)

sales.fillna(0, inplace=True)    
sales.info()

In [ ]:
# Selling Trend feature
sales['daily_avg_units'] = sales.groupby(['id','d'])['units_sold'].transform('mean').fillna(0)
sales['avg_units'] = intermediate_df.transform('mean').fillna(0)

sales['selling_trend'] = (sales['daily_avg_units'] - sales['avg_units'])

sales.drop(['daily_avg_units','avg_units'],axis=1,inplace=True)
sales.info()

In [ ]:
# Mean Encoding features
sales['mean_units_by_item'] = df.groupby('item_id',as_index=False, sort=False)['units']\
                                                     .transform('mean')\
                                                     .astype(np.float16)
sales['mean_units_by_state'] = df.groupby('state_id',as_index=False, sort=False)['units']\
                                                      .transform('mean')\
                                                      .astype(np.float16)
sales['mean_units_by_store'] = df.groupby('store_id',as_index=False, sort=False)['units']\
                                                      .transform('mean')\
                                                      .astype(np.float16)
sales['mean_units_by_cat'] = df.groupby('cat_id',as_index=False, sort=False)['units']\
                                                    .transform('mean')\
                                                    .astype(np.float16)
sales['mean_units_by_dept'] = df.groupby('dept_id',as_index=False, sort=False)['units']\
                                                     .transform('mean')\
                                                     .astype(np.float16)

sales['mean_units_by_cat_dept'] = df.groupby(['cat_id','dept_id'],as_index=False, sort=False)['units']\
                                                         .transform('mean')\
                                                         .astype(np.float16)
sales['mean_units_by_store_item'] = df.groupby(['store_id','item_id'],as_index=False, sort=False)['units']\
                                                           .transform('mean')\
                                                           .astype(np.float16)
sales['mean_units_by_cat_item'] = df.groupby(['cat_id','item_id'],as_index=False, sort=False)['units']\
                                                         .transform('mean')\
                                                         .astype(np.float16)
sales['mean_units_by_dept_item'] = df.groupby(['dept_id','item_id'],as_index=False, sort=False)['units']\
                                                          .transform('mean')\
                                                          .astype(np.float16)
sales['mean_units_by_state_store'] = df.groupby(['state_id','store_id'],as_index=False, sort=False)['units']\
                                                            .transform('mean')\
                                                            .astype(np.float16)

sales['mean_units_by_state_store_cat'] = df.groupby(['state_id','store_id','cat_id'],as_index=False, sort=False)['units']\
                                                                .transform('mean')\
                                                                .astype(np.float16)
sales['mean_units_by_store_cat_dept'] = df.groupby(['store_id','cat_id','dept_id'],as_index=False, sort=False)['units']\
                                                               .transform('mean')\
                                                               .astype(np.float16)

sales.fillna(0, inplace=True)
sales.info()

In [ ]:
# Custom Metric function - RMSSE

def RMSE(actual,predictions):
    if type(actual) != 'numpy.ndarray':
        actual = np.array(actual)
    if type(predictions) != 'numpy.ndarray':
        predictions = np.array(predictions)
    assert actual.shape[0] == predictions.shape[0], "Observation count Mismatched...!"
    assert len(actual.shape) == 1 and len(predictions.shape) == 1, "RMSE takes single dimension list..."
    
    return round(np.sqrt(((actual-predictions)**2).mean()),3)

def RMSSE(y_actual, y_pred, train_series, axis=1):
    assert axis == 0 or axis == 1
    if type(y_actual) != 'numpy.ndarray': y_actual = np.array(y_actual)
    if type(y_pred) != 'numpy.ndarray': y_pred = np.array(y_pred)
    if type(train_series) != 'numpy.ndarray': train_series = np.array(train_series)
    
    assert y_actual.shape == y_pred.shape
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert y_actual.shape[1] > 1 and y_pred.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((y_actual - y_pred)**2).sum(axis=axis)
    n = train_cols
    
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return ((1/forcasting_horizon) * numerator/denominator) ** 0.5


In [ ]:
## Utility Functions
# Submission Format df
def submission(X_test, y_test, predictions):
    """
    >> submission(X_test, y_test, predictions)
    Inputs:
        i)   X_test must have 2 columns: 
                -> 'id' : integer values id's which will be used for getting mapped product ids
                -> 'd'  : Day num columns with integer values only
        ii)  y_test is original answers for ['id','d'] combinations
        iii) predictions are pred values
    
    Returns:
        df in final submission format
    """
    # Getting just important cols
    df = X_test[['id','d']]
    df['actual'] = y_test
    df['prediction'] = predictions
    
    # F_ day formatted columns
    start = df.d.min() - 1
    df['d'] = df['d'] - start
    
    # Pivot and Id string mappings
    id_map = joblib.load('./pre-processed/encoded/map_id.pkl')
    
    valid_df = pd.pivot(df,index='id',values ='prediction',columns = 'd').reset_index()
    valid_df['id'] = valid_df['id'].map(id_map).str.replace('evaluation','validation')
    
    eval_df = pd.pivot(df,index='id',values ='actual',columns = 'd').reset_index()
    eval_df['id'] = eval_df['id'].map(id_map)
    
    results = pd.concat([valid_df,eval_df])
    results = results.set_index('id').add_prefix('F').reset_index()
    del valid_df
    del eval_df
    
    return results
    

# To get RandomizedSearchCV with TimeSplit indexes provided
def get_RS_model(estimator=None, param_distributions=None ,n_iter=3,n_jobs=1):
    """
    Get the RandomizedSearchCV model object with 3-Fold TimeSplit of X_train data with Default 3 Randomized sampled parameters from each HyperParameter
    Inputs: 1) estimator 2) param_distributions for given estimator
    Output: RandomizedSearch 
    """
    return  RandomizedSearchCV(estimator, param_distributions, cv=train_test_idx,\
                               n_iter=n_iter, scoring='neg_mean_squared_error', n_jobs=1,random_state=random_state,\
                               refit=False, verbose=2, return_train_score=True)

# To get feature importance bar Chart
def plot_feature_importance(model = None, test_df = None, skip = 0):
    """
    >>> Usage: plot_feature_importance(model = estimator, test_df = X_test, skip = 0)
    skip parameter is used when the number of features are large and you want leave out/skip the least k important points in graph.
    """
    features = test_df.columns
    if hasattr(model,'coef_'):
        importances = model.coef_
    
    elif hasattr(model,'feature_importances_'):
        importances = model.feature_importances_
    if skip != 0:
        indices = np.argsort(importances)[skip:]
    else:
        indices = np.argsort(importances)
        
    plt.figure(figsize=(6,8))
    plt.barh(range(len(indices)) , importances[indices])
    
    plt.title('Feature Importance',fontsize=14)
    plt.xlabel('Relative Importance',fontsize=14)
    plt.yticks(range(len(indices)),[features[i] for i in indices])
    plt.show()

In [ ]:
joblib.dump(sales,processed_fpath+'featured_data.pkl')
del sales

In [ ]:
# importing featured data from Phase-3.1 
data = joblib.load(processed_fpath+'featured_data.pkl')
data.info()

In [ ]:
# Train-Val-Test set split
X_train = data.loc[data.d < 1886]
y_train = X_train['units_sold']
X_train.drop(['units_sold'], axis=1, inplace=True)

# Validation dataset
validation_days=np.arange(1886,1914)
X_val = data.loc[data.d.isin(validation_days)]
y_val = X_val['units_sold']
X_val.drop(['units_sold'], axis=1, inplace=True)

# Test set
X_test  = data.loc[data.d > 1913]
y_test  = X_test['units_sold']
X_test.drop(['units_sold'], axis=1, inplace=True)

print("X_train: {0} \t y_train: {1}\n".format(X_train.shape,y_train.shape))
print("X_val:\t {0}\t\t y_val:\t {1}\n".format(X_val.shape,y_val.shape))
print("X_test:\t {0}\t\t y_test: {1}\n".format(X_test.shape,y_test.shape))

# For space issues
del data

In [ ]:
from lightgbm import LGBMRegressor

# Getting best model fit for predictions
model = LGBMRegressor(learning_rate=0.035,\
                      max_depth=148,\
                      num_leaves=375,\
                      lambda_l2=0.05,\
                      n_estimators=100, random_state = random_state)

for k in range(len(train_test_idx)):
    train_idx, val_idx = train_test_idx[k]
    X = X_train.iloc[train_idx]
    y = y_train.iloc[train_idx]
    model.fit(X,y)
    cv_hat = model.predict(X_train.iloc[val_idx])
    print("CV-{0} Error: {1}".format(k+1,RMSE(y_train.iloc[val_idx],cv_hat)))

# Validation score
Y_hat = model.predict(X_val)
print("\nValidation Error: ",RMSE(y_val,Y_hat))

model.fit(X_val,y_val)
# Test scores
y_pred = model.predict(X_test)
print("\nTest Error: ",RMSE(y_test,y_pred))

In [ ]:
# Plotting Feature Importances
plot_feature_importance(model = model, test_df = X_test,skip=8)

In [ ]:
# Get Submission Format df
sub_df = submission(X_test, y_test, predictions = y_pred)
sub_df

In [ ]:
# Saving df and Model
sub_df.to_csv(submission_dir+'submission_LightGBM.csv',index=False)
joblib.dump(model,models_dir+'Model_LightGBM.pkl')

In [ ]:
# Cleanup 
cleanup_list = [sub_df, Y_hat, y_pred, model, X, y, cv_hat]
for obj in cleanup_list:
    del obj
     